In [3]:
# Just run this block. Please do not modify the following code.
import math
import time
import io
import numpy as np
import csv
from IPython.display import Image

# Pytorch package
import torch
import torch.nn as nn
import torch.optim as optim

# Torchtest package
import torchtext
from torchtext.datasets import Multi30k
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab
from torchtext.utils import download_from_url, extract_archive
from torch.nn.utils.rnn import pad_sequence

# Tqdm progress bar
from tqdm import tqdm_notebook, tqdm

# Code provide to you for training and evaluation

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

torch.seed

/opt/homebrew/anaconda3/envs/cs7643-finalproject/lib/python3.9/site-packages/torchtext/datasets/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/opt/homebrew/anaconda3/envs/cs7643-finalproject/lib/python3.9/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/opt/homebrew/anaconda3/envs/cs7643-finalproject/lib/python3.9/site-pack

In [70]:
# Check device availability
device = ''
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
device = torch.device(device)
print("You are using device: %s" % device)

You are using device: mps


In [71]:
encoder_output = torch.rand((5, 320, 20)).to(device) # N, input_size, 1

In [72]:
from Decoder import Decoder
from PostNet import PostNet

In [73]:
decoder = Decoder().to(device)
decoder_output = decoder.forward(encoder_output)
print(decoder_output.shape) # N, output_size, 1
print(decoder_output)

torch.Size([5, 80, 20])
tensor([[[ 0.0224,  0.0188,  0.0195,  ...,  0.0067,  0.0096,  0.0167],
         [ 0.0114,  0.0210,  0.0291,  ...,  0.0029, -0.0004, -0.0080],
         [ 0.0259,  0.0196,  0.0217,  ...,  0.0188,  0.0191,  0.0229],
         ...,
         [-0.0291, -0.0313, -0.0327,  ..., -0.0244, -0.0204, -0.0261],
         [ 0.0204,  0.0196,  0.0206,  ...,  0.0259,  0.0224,  0.0254],
         [-0.0081,  0.0020,  0.0061,  ...,  0.0009,  0.0007, -0.0047]],

        [[ 0.0179,  0.0170,  0.0144,  ..., -0.0069, -0.0062,  0.0070],
         [ 0.0045,  0.0113,  0.0083,  ...,  0.0128,  0.0067,  0.0025],
         [ 0.0200,  0.0147,  0.0127,  ...,  0.0186,  0.0237,  0.0313],
         ...,
         [-0.0332, -0.0342, -0.0299,  ..., -0.0250, -0.0240, -0.0261],
         [ 0.0246,  0.0209,  0.0208,  ...,  0.0210,  0.0304,  0.0300],
         [-0.0107, -0.0039, -0.0034,  ...,  0.0002,  0.0028,  0.0042]],

        [[ 0.0167,  0.0162,  0.0072,  ...,  0.0237,  0.0288,  0.0335],
         [ 0.0081,  0

In [74]:
postnet = PostNet().to(device)
postnet_output = postnet.forward(decoder_output)
print(postnet_output.shape) # (N, output size, 1)
print(postnet_output)

torch.Size([5, 80, 20])
tensor([[[-1.4014e-01, -2.9221e-01, -1.0126e-01,  ...,  4.0079e-01,
           3.6558e-01, -2.0843e-01],
         [-6.3166e-02, -3.6170e-01, -4.7774e-01,  ..., -2.8799e-01,
          -3.0284e-01, -4.6196e-01],
         [-3.0782e-01,  4.4426e-01,  2.6556e-01,  ...,  3.2749e-02,
          -3.9405e-01, -4.2490e-01],
         ...,
         [-9.1223e-01, -5.0052e-01, -8.6679e-02,  ..., -5.3131e-01,
          -2.1090e-01, -5.0512e-01],
         [-9.3360e-03, -5.8920e-01, -3.8786e-01,  ..., -2.6266e-01,
          -7.5875e-03, -1.8152e-01],
         [ 1.1962e-01,  6.5525e-03,  8.9589e-02,  ...,  5.6906e-01,
           2.6966e-01,  2.7719e-01]],

        [[-2.7390e-01, -6.8670e-01, -1.5890e-01,  ...,  4.0504e-01,
           9.9523e-02, -2.9345e-01],
         [-2.6868e-01, -6.9413e-01, -5.0518e-01,  ..., -8.5273e-01,
          -1.4182e-01, -1.9727e-01],
         [-2.3454e-01,  6.2080e-01,  4.7377e-01,  ..., -7.1088e-01,
          -3.4671e-01, -9.6181e-02],
         ...,
 

In [75]:
# Loss Test
from utils import criterion

input = torch.rand((5, 80, 1)).to(device)

criterion(input=input, encoder_ouput=encoder_output, decoder_output=decoder_output, postnet_output=postnet_output, encoded_postnet_output=encoder_output)

/opt/homebrew/anaconda3/envs/cs7643-finalproject/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([5, 80, 20])) that is different to the input size (torch.Size([5, 80, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.8126, device='mps:0', grad_fn=<AddBackward0>)